## Hidden Markov Model Study

Goal: Revisit Hidden Markov Models from the start, write slow but readable code. Build a Hidden Markov Model from scratch.

Stretch goals: 
- revisit Theoretic foundations for Multivariate Categorically distributed emissions.
- how does the independence assumptions for multivariate emissions change the theory?


## In the Defense of Simple Models

Motivating Hidden Markov Models is hard without asking some fundamental questions about statistical and probabilistic inference and learning.

Why do we model things? What are we interested about finding out - the underlying dynamics or solely predictions into the future? Do we even want to predict into the future or are we satisfied with estimating some underlying state responsible for our observations?
Why not choose Timeseries model type X over type Y? Well, there is no easy answer to this, it may just be that another type of model is better fit for a specific requirement.

Markov Models and *Hidden* Markov Models are so-called **generative** models. This means that after building them through a process called *training* or *fitting*, we may produce new pseudo-data from our model. We may extrapolate from timepoint $t$ onwards or simulate a new trajectory. We may choose different initial conditions and see, how the model evolves. It is here, i want to make an argument for Hidden Markov Models in the age of Neural Networks. 
The biggest advantage of HMMs over Neural Methods is in my understanding, that they are **interpretable**. They are white-box models, allowing the observer to clearly understand **why** the model has produced any output. To some degree, the data **is** the model. All important information has been distilled and transformed into a model.

Another big advantage is that they are **simple** models. No-hyperparamter-tuning. No tinkering with the learning rate or swapping out optimizers, adding dropout, adding layer norms, removing layer norms... The process of fitting the model is more sane. In some sense this is a half-truth, as we do have to make assumptions when we choose a model type, however they are very different types of assumptions. They encourage us to think harder about the problem and the setup, not decide on model changes depending on whether or not the $F_1$-Score is moving up or down.

## Definitions.

Let a Hidden Markov Model be a tuple $\lambda = (\mathcal A, \mathcal B, \pi)$ where 

\begin{align*}
    \mathcal A := p(x_{t}|X_{0:t-1})\quad & \text{Conditional distribution over hidden state given previous hidden states}\\
    \mathcal B := p(y_{t}|X_{0:t})\quad& \text{Conditional distribution over emission signal given current and previous hidden states}\\ 
    \pi := p(x_0)\quad & \text{Initial state distribution}
\end{align*}

In my thesis and the following study, we will look at a special case of HMMs, where $\mathcal A, \mathcal B$ and $\pi$ are categorical distributions, making them row-stochastic matrices containing transition and emission probabilities. The probabilistic formulation of the object lets us see however, how we might use **Bayes-Rule** to do inference on the hidden states, as

$$
    p(X_{0:t}|y_t) = \frac{p(y_t|X_{0:t})p(X_{0:t})}{p(y_t)} \quad \text{ or } \quad p(X_{0:N}| Y_{0:N}) = \frac{p(Y_{0:N}|X_{0:N})p(X_{0:N})}{p(Y_{0:N})}
$$

Calculating these expressions however is untractable, as it would require us to compute large integrals $p(Y) = \int p(Y|X)p(X) dX$ where $p(X) = p(x_0)p(x_1|x_0)\dots p(x_N|x_0, \dots, x_{N-1})$ itself would be tricky to compute. It is here where we make **simplifying independence assumptions** called the *Markov Assumption*:

$$
    p(x_t|x_0, x_1, \dots x_{t-1}) = p(x_t|x_{t-1}) \quad \text{and} \quad p(y_t|x_0, x_1, \dots, x_{t}) = p(y_t|x_t)
$$

In words, we assume that transitions only depend on the previously visited state and that emissions only depend on the current hidden state. This simplifies the computations, as we may factor expressions as $p(X)$ into parts. 



### 

## Introductory Example

We are concerned with making statements about any **hidden** or **latent** variable $x$. In this example, lets consider the following setup: You're sitting at home, listening to the weather news on the radio. The true weather however is not-observable, as your window blinds are down. In our model, the true weather will be the non-observable (i.e. the *hidden* variable $x_t$) and the news will be the observable variable $y_t$. There are only two different 'true' weather states: Sunny $S$ and Rainy $R$, whereas the media reports three different observations: Dry $D$, Wet $W$ and Cloudy $C$.
The prior over the hidden states is a discrete uniform distribution. Additionally, we **know** $\mathcal A, \mathcal B$. 

**IMPORTANT** This is obviously a shortcut. In reality we dont know $\mathcal A$ and $\mathcal B$, we have to fit them, using our data! Notice, how our motivation has changed the model and how we use it. Instead of being interested *how exactly* the weather changes or in which hidden state what wheather observation is made, we focus on prediction rather than trying to understand underlying dynamics.

We summarize the assumptions with these equations:

\begin{align*}
    \mathcal A &= \begin{bmatrix}
        P(S|S) & P(R|S)\\
        P(S|R) & P(R|R)
    \end{bmatrix} = \begin{bmatrix}
        0.7 & 0.3\\
        0.2 & 0.8
    \end{bmatrix}\\
    \mathcal B &= \begin{bmatrix}
        P(D|S) & P(W|S) & P(C|S)\\
        P(D|R) & P(W|R) & P(C|R)
    \end{bmatrix} = \begin{bmatrix}
        0.5 & 0.3 & 0.2\\
        0.0 & 0.8 & 0.2
    \end{bmatrix}\\
    \pi &= \begin{bmatrix}
        P(x_0 = S)\\
        P(x_0 = R)
    \end{bmatrix} = \begin{bmatrix}
        0.5\\
        0.5
    \end{bmatrix}
\end{align*}

### Predicting and Updating

Inference on timeseries is usually separated into three steps: **predicting** the next hidden state $\tilde x = p(x_t| X_{0:t-1}, Y_{0:t-1})$ and **updating** said prediction once we have observed the current emission $\hat x = p(x_t | X_{0:t-1}, Y_{0:t-1}, y_t)$

The recursive prediction equation is:

$$
    p(x_t| Y_{0:t-1}) = \int \underbrace{p(x_t|x_{t-1})}_{\text{hidden state dynamics}}\cdot \overbrace{p(x_{t-1}|Y_{0:t-1})}^{\text{previous update result} } dx_{t-1}  
$$

This is a local integral over $x_{t-1}$, which we may be able to solve, if we have access to the previous update result, which is similarily recursively defined as:

$$
    p(x_t|Y_{0:t}) = \frac{\overbrace{p(y_t|x_t)}^{\text{likelihood}}\overbrace{p(x_t|Y_{0:t-1})}^{\quad \text{prev. prediction result}}}{\int p(y_t | x_t )p(x_t | Y_{0:t−1}) dx_t}
$$

We can see, that we have to compute the prediction and the update recursively, in order to get an updated estimate of the distribution over $x_t$.

### Calculation

Now, how would we go about calculating these quantities? We should start at the beginning, calculating the prediction of $x_1$, before updating it with our observation. For the sake of this eaxmple, lets assume we've observed the sequence $Y_{0:3} = \{ D, C, C, W\}$ and the state and emission matrices are as described above.

Calculate the update for $t=0$:

\begin{align*}
    p(x_0) &= \pi\\
    p(x_0|y_0) &= \frac{p(y_0|x_0)p(x_0)}{\int p(y_0|x_0)p(x_0) dx_0} = \frac{\begin{bmatrix}
        P(D|S)\\
        P(D|R)
    \end{bmatrix}\odot \begin{bmatrix}
        P(S)\\
        P(R)
    \end{bmatrix}}{P(D|S)P(S) + P(D|R)P(R)}\\
    &= \frac{\begin{bmatrix}
        0.5\\
        0.0
    \end{bmatrix}\odot \begin{bmatrix}
        0.5\\
        0.5
    \end{bmatrix}}{0.5 \cdot 0.5 + 0.0 \cdot 0.5} = \begin{bmatrix}
        1\\
        0
    \end{bmatrix}
\end{align*}\\


Seems like after seeing the first observation $y_0 = D$, we are completely convinced, that it is sunny outside. Lets see, what happens, if we predict the next hidden state.

\begin{align*}
    p(x_0|y_0) &= \begin{bmatrix}
        1\\
        0
    \end{bmatrix}\\
    p(x_1|x_0, y_0) &= \int p(x_1|x_0)p(x_0|y_0)dx_0\\
    &= \mathcal A ^T p(x_0|y_0)\\
    &= \begin{bmatrix}
        0.7\\
        0.3
    \end{bmatrix}
\end{align*}

Finally, lets update for our estimate of the hidden state at $t=1$. For this, we calculate the update analogously to our update at timestep $t= 0$, with the difference, that now we use the posterior of the prediction as the prior.

\begin{align*}
    p(x_1) &= p(x_1|y_0, x_0) = \begin{bmatrix}
    0.7\\
    0.3
    \end{bmatrix}\\
    p(x_1|Y_{0:1}) &= \frac{\mathcal B[:, 2] \odot p(x_1|y_0, x_0)}{\sum_i \mathcal B[:, 2] \odot p(x_1|y_0, x_0)}\\
    &= \frac{\begin{bmatrix}
        0.2\\
        0.2
    \end{bmatrix}\odot \begin{bmatrix}
        0.7\\
        0.3
    \end{bmatrix}}{0.2 \cdot 0.7 + 0.2 \cdot 0.3}\\
    &= \begin{bmatrix}
        0.7\\
        0.3
    \end{bmatrix}
\end{align*}\\



We will put this prediction and update routine in code below:

In [4]:
import numpy as np

A = np.array([
    [0.7, 0.3],
    [0.2, 0.8]
])

B = np.array([
    [0.5, 0.2, 0.3],
    [0.0, 0.2, 0.8]
])

x_0 = np.array([0.5, 0.5])

In [12]:
def update(x_tm1 : np.ndarray, y_t : int,  B : np.ndarray) -> np.ndarray:

    likelihood = B[:, y_t]
    prior = x_tm1

    posterior_unnormalized = likelihood * prior
    return posterior_unnormalized / posterior_unnormalized.sum()

def predict(x_tm1 : np.ndarray, A : np.ndarray) -> np.ndarray:

    # check for row-stochasticity
    assert np.all(np.isclose(A.sum(axis=1), 1)), "Matrix A doesn't seem to be row-stochastic"

    #check if x_tm1 is a valid probability distribution
    assert np.isclose(x_tm1.sum(), 1), "x_tm1 isn't stochastic, sum != 1"

    return A.T @ np.atleast_1d(x_tm1)


In [11]:
observations = [0, 1, 1, 2] # [D, C, C, W]
x_tm1 = x_0
predictions = [x_0]
updates = []

for y_i in observations:
    x_updated = update(x_tm1, y_i, B)
    x_tm1 = predict(x_updated, A)

    updates.append(x_updated)
    predictions.append(x_tm1)

    
predictions, updates

([array([0.5, 0.5]),
  array([0.7, 0.3]),
  array([0.55, 0.45]),
  array([0.475, 0.525]),
  array([0.32666667, 0.67333333])],
 [array([1., 0.]),
  array([0.7, 0.3]),
  array([0.55, 0.45]),
  array([0.25333333, 0.74666667])])

### Multivariate Case

Before looking at smoothing, let us think about changing the likelihood function $p(y_t | x_t)$. What would change, if y_t would be a collection of $K$ independent categorically distributed random variables instead of a single random variable? We would have to keep track of $K$ different emission matrices and calculate the update differently, as $p(y_t|x_t)$ now expands into $\prod_{i = 1}^K p(y^i_t | x_t)$.

We extend our example of the TV and weather with another observation variable. Lets assume we would observe dogs barking depending on the weather. If it's sunny, people much rather go out to walk their dog than if its raining. From this assumption follows the emission matrix $\mathcal B_2 = \begin{bmatrix}
    P(B|S) & P(\neg B|S)\\
    P(B|R) & P(\neg B|R)
\end{bmatrix} = \begin{bmatrix}
    0.6 & 0.4\\
    0.1 & 0.9
\end{bmatrix}$

In [13]:
A = np.array([
    [0.7, 0.3],
    [0.2, 0.8]
])

B_1 = np.array([
    [0.5, 0.2, 0.3],
    [0.0, 0.2, 0.8]
])

B_2 = np.array([
    [0.6, 0.4],
    [0.1, 0.9]
])

x_0 = np.array([0.5, 0.5])

In [18]:
from typing import Sequence

In [19]:
def update(x_tm1 : np.ndarray, y_t : Sequence[int],  Bs : Sequence[np.ndarray]) -> np.ndarray:

    """
        x_tm1   : np.ndarray        vector of hidden state distribution from timestep t-1
        y_t     : Sequence[int]     Sequence of ints, int at index i contains observation for corresponding emission matrix at index i
        Bs      : Sequence[np.ndarray]  Sequence of emission matrices              
    """

    # likelihood p(y_t | x_t) factorizes into p(y^1_t|x_t) * p(y^2_t|x_t) * ... *  p(y^K_t|x_t)
    likelihoods = np.ones_like(x_tm1)
    for y, B in zip(y_t, Bs):
        likelihoods *= B[:, y]
    prior = x_tm1
    
    posterior_unnormalized = likelihoods * prior
    return posterior_unnormalized / posterior_unnormalized.sum()

def predict(x_tm1 : np.ndarray, A : np.ndarray) -> np.ndarray:

    # check for row-stochasticity
    assert np.all(np.isclose(A.sum(axis=1), 1)), "Matrix A doesn't seem to be row-stochastic"

    #check if x_tm1 is a valid probability distribution
    assert np.isclose(x_tm1.sum(), 1), "x_tm1 isn't stochastic, sum != 1"

    return A.T @ np.atleast_1d(x_tm1)


In [20]:
observations = [[0, 1], [1, 0], [1, 1], [1, 0]] # (D, not B), (C, B), (C, not B), (W, B)
B = [B_1, B_2]
x_tm1 = x_0
predictions = [x_0]
updates = []

for y_i in observations:
    x_updated = update(x_tm1, y_i, B)
    x_tm1 = predict(x_updated, A)

    updates.append(x_updated)
    predictions.append(x_tm1)

    
predictions, updates


([array([0.5, 0.5]),
  array([0.7, 0.3]),
  array([0.66666667, 0.33333333]),
  array([0.43529412, 0.56470588]),
  array([0.61111111, 0.38888889])],
 [array([1., 0.]),
  array([0.93333333, 0.06666667]),
  array([0.47058824, 0.52941176]),
  array([0.82222222, 0.17777778])])